# Gauge R&R Experiments in R

First input data for the thermal impedance data from Montgomery:

In [47]:
# thermal impedance data from montgomery
data.wide <- matrix(c(
    37, 38, 37, 41, 41, 40, 41, 42, 41,
    42, 41, 43, 42, 42, 42, 43, 42, 43,
    30, 31, 31, 31, 31, 31, 29, 30, 28,
    42, 43, 42, 43, 43, 43, 42, 42, 42,
    28, 30, 29, 29, 30, 29, 31, 29, 29,
    42, 42, 43, 45, 45, 45, 44, 46, 45,
    25, 26, 27, 28, 28, 30, 29, 27, 27,
    40, 40, 40, 43, 42, 42, 43, 43, 41,
    25, 25, 25, 27, 29, 28, 26, 26, 26,
    35, 34, 34, 35, 35, 34, 35, 34, 35), nrow = 10, ncol = 9, byrow = TRUE)

data.wide

37,38,37,41,41,40,41,42,41
42,41,43,42,42,42,43,42,43
30,31,31,31,31,31,29,30,28
42,43,42,43,43,43,42,42,42
28,30,29,29,30,29,31,29,29
42,42,43,45,45,45,44,46,45
25,26,27,28,28,30,29,27,27
40,40,40,43,42,42,43,43,41
25,25,25,27,29,28,26,26,26
35,34,34,35,35,34,35,34,35


The data is presented in the textbook in wide format. We reformat using the `reshape()` function.

In [48]:
data.wide.1 <- data.frame(part = 1:10, inspector = "1", data.wide[,1:3])
data.wide.2 <- data.frame(part = 1:10, inspector = "2", data.wide[,4:6])
data.wide.3 <- data.frame(part = 1:10, inspector = "3", data.wide[,7:9])

data.wide.4 <- rbind(data.wide.1,data.wide.2,data.wide.3)
colnames(data.wide.4)[3:5] <- c("test1","test2","test3")

data.long <- reshape(data.wide.4, 
        direction = "long",
        varying = list(names(data.wide.4)[3:5]),
        v.names = "impedance",
        timevar = "test",
        times = 1:3)

rownames(data.long) <- NULL

head(data.long, 10)

part,inspector,test,impedance,id
1,1,1,37,1
2,1,1,42,2
3,1,1,30,3
4,1,1,42,4
5,1,1,28,5
6,1,1,42,6
7,1,1,25,7
8,1,1,40,8
9,1,1,25,9
10,1,1,35,10


First we demonstrate how this can be done using a simple ANOVA with the `aov` function in `R`.

In [49]:
process.aov <- aov(impedance ~ factor(inspector) + factor(part) + factor(inspector) * factor(part),
                  data = data.long)

summary.aov(process.aov)

                               Df Sum Sq Mean Sq F value   Pr(>F)    
factor(inspector)               2     39    19.6  38.413 1.82e-11 ***
factor(part)                    9   3936   437.3 855.643  < 2e-16 ***
factor(inspector):factor(part) 18     49     2.7   5.273 5.06e-07 ***
Residuals                      60     31     0.5                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Estimating the variance components are easier done using the `VCAinference` function in the `VCA` package in `R`.

In [50]:
library(VCA)

data.long$inspector <- as.factor(data.long$inspector)
data.long$part <- as.factor(data.long$part)

process.vca <- fitVCA(impedance ~ inspector + part + inspector * part,
                  data.long)

VCAinference(process.vca, VarVC=TRUE)




Inference from (V)ariance (C)omponent (A)nalysis
------------------------------------------------

> VCA Result:
-------------

  Name           DF     SS        MS       VC      %Total  SD     CV[%]  
1 total          9.5575                    50.0963 100     7.0779 19.7706
2 inspector      2      39.2667   19.6333  0.5646  1.127   0.7514 2.0989 
3 part           9      3935.9556 437.3284 48.2926 96.3995 6.9493 19.4114
4 inspector:part 18     48.5111   2.6951   0.728   1.4532  0.8532 2.3833 
5 error          60     30.6667   0.5111   0.5111  1.0203  0.7149 1.997  
  Var(VC) 
1         
2 0.4292  
3 524.7181
4 0.0906  
5 0.0087  

Mean: 35.8 (N = 90) 

Experimental Design: balanced  |  Method: ANOVA


> VC:
-----
               Estimate CI LCL  CI UCL   One-Sided LCL One-Sided UCL
total          50.0963  24.1327 159.4836 27.0575       130.622      
inspector      0.5646   0*      1.8486   0*            1.6422       
part           48.2926  3.3962  93.189   10.6144       85.9708     

In Montgomery, we know that $\sigma^2 = 0.5111$ is the repeatability component of the Gauge R&R Study. Meanwhile, $\sigma^2_{Reproducibility} = \sigma^2_{O} + \sigma^2_{PO}$:

In [56]:
sigsq.repeat = 0.511
sigsq.reproduce = 0.5646 + 0.728

sigsq.gauge = sigsq.repeat + sigsq.reproduce
sigsq.gauge

[1] 1.8036

In [58]:
sqrt(sigsq.gauge)

[1] 1.342982

Supposing that we set LSL = 18 and USL = 58, we can produce the P/T ratio using $\sigma_{Gauge}$:

In [61]:
pt = (6*sqrt(sigsq.gauge))/(58 - 18)
pt

[1] 0.2014473

Recall that we generally prefer PT ratio to not exceed 0.10.